# Regex Matching (. and * only)

- JZ Offer problem 19
- LeetCode CN：https://leetcode.cn/problems/zheng-ze-biao-da-shi-pi-pei-lcof/

## General idea

1. not p not s -> true
2. not p -> false
3. p has * (c* or .*)
   1. match none, or
   2. match match, or
   3. match many
4. p is just c or .
   1. not s -> false
   2. s[0] == p[0] -> check remaining
   3. s[0] != p[0] -> false

Below is a faithful implementation of the above idea. It uses `functools.cache` to make it dynamic programming.

In [22]:
from functools import cache

@cache
def match(s, p):
  if not s and not p:
    return True
  if not p:
    return False
  if not (len(p) > 1 and p[1] == '*'):  # pattern is NOT "c*" or ".*"
    if not s:
      return False
    if p[0] != '.':  # pattern is "c"
      if p[0] == s[0]:
        return match(s[1:], p[1:])
    else:  # pattern is "."
      return match(s[1:], p[1:])
  else:  # pattern IS "c*" or ".*"
    match_none = match(s, p[2:])
    match_at_least_one = bool(s and (p[0] == s[0] or p[0] == '.'))
    match_one = match_at_least_one and match(s[1:], p[2:])
    match_many = match_at_least_one and match(s[1:], p)
    return match_none or match_one or match_many
  return False

A cleaner, simplified but less readable version:

In [50]:
from functools import cache

@cache
def match(s, p):
  if not p: return not s
  if len(p) > 1 and p[1] == '*':
    return match(s, p[2:]) or \
      bool(s and (p[0] == '.' or p[0] == s[0])) \
      and (match(s[1:], p[2:]) or match(s[1:], p))
  if s and (p[0] == '.' or p[0] == s[0]):
    return match(s[1:], p[1:])
  return False

Here is a test block you can run to check the correctness.

In [51]:
assert match('abc', 'abc') is True
assert match('abc', 'a.c') is True
assert match('', '') is True
assert match('a', '.') is True
assert match('aaa', 'a*aa') is True
assert match('aa', 'a*aa') is True
assert match('a', 'a*') is True
assert match('a', 'a*aa') is False
assert match('a', 'a*b') is False
assert match('ab', '.*') is True
assert match('ab', '.*c') is False
assert match('mississippi', 'mis*is*p*.') is False